In [23]:
from scipy.optimize import linprog

# a) 
# Digitos GRR
X = 6
Y = 4
Z = 1

PC = 2100 + (Z * 10) # Preco de venda de C por tonelada em reais
# Funcao objetivo
Fobj = (- PC + 2000, - PC + 1500) # Custos e preco de venda
# Inequacoes
# lado direito
LD = [[-0.04, 0.07], # inequacao 1 Carbono -4% * x + 7% * y <= 0 
      [-0.005, 0.005], # inequacao 2 Enxofre -0.5% * x + 0.5% * y <= 0
      [-0.03, Y * 1e-2], # inequacao 3 cinzas -3% * x + Y% * y <= 0
      [1, 1]] # inequacao 4 demanda - x - y <= - 20 - X  
# lado esquerdo
LE = [0, # inequacao 1 Carbono -4% * x + 7% * y <= 0 
      0, # inequacao 2 Enxofre -0.5% * x + 0.5% * y <= 0 
      0, # inequacao 3 cinzas -3% * x + Y% * y <= 0
      20 + X] # inequacao 4 demanda  - x - y <= - 20 - X
# Optimizacao por Simplex ++ 
opt = linprog(c = Fobj, A_ub = LD, b_ub = LE)
print(f"A:{opt.x[0]:.5f} B:{opt.x[1]:.5f}, Lucro: {opt.fun:.5f}")
opt = linprog(c = Fobj, A_ub = LD, b_ub = LE, integrality = 1)
print(f"A:{opt.x[0]} B:{opt.x[1]}, Lucro: {opt.fun}")

A:16.54545 B:9.45455, Lucro: -7587.27273
A:17.0 B:9.0, Lucro: -7360.0


In [24]:
import pulp
# b)

C1 = pulp.LpProblem(name="C1", upBound = 3000)
C2 = 
Obj = [13, 15.30, 14.60, 14.90, # C1, C2, C3, C4
       -16.20, -15.75, -15.30] # A, B, C

# Inequacoes
# lado direito
LD = [[0.85, -0.15, -0.15, -0.15, 0, 0, 0], # inequacao 1 85% * C1 - 15% * C2 - 15% * C3 - 15% * C4 <= 0 
      [0.4, -0.6, 0.4, 0.4, 0, 0, 0], # inequacao 2 40% * C1 - 60% * C2 + 40% * C3 + 40% * C4 <= 0
      [-0.5, -0.5, 0.5, -0.5, 0, 0, 0], # inequacao 3 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      [-0.5, -0.5, 0.5, -0.5, 0, 0, 0], # inequacao 4 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      [0.9, -0.1, -0.1, -0.1, 0, 0, 0], # inequacao 5 90% * C1 - 10% * C2 + 10% * C3 - 10% * C4 <= 0 
      [0.1, -0.9, 0.1, 0.1, 0, 0, 0], # inequacao 6 10% * C1 - 90% * C2 + 10% * C3 + 10% * C4 <= 0
      [0.8, -0.2, -0.2, -0.2, 0, 0, 0] # inequacao 7 80% * C1 - 20% * C2 - 20% * C3 - 20% * C4 <= 0 
      ] 
# lado esquerdo
LD = [0, # inequacao 1 85% * C1 - 15% * C2 - 15% * C3 - 15% * C4 <= 0 
      0, # inequacao 2 40% * C1 - 60% * C2 + 40% * C3 + 40% * C4 <= 0
      0, # inequacao 3 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      0, # inequacao 4 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      0, # inequacao 5 90% * C1 - 10% * C2 + 10% * C3 - 10% * C4 <= 0 
      0, # inequacao 6 10% * C1 - 90% * C2 + 10% * C3 + 10% * C4 <= 0
      0 # inequacao 7 80% * C1 - 20% * C2 - 20% * C3 - 20% * C4 <= 0 
      ] 
# Contorno
CC = [(0, 3000), # C1 max 
      (0, 2000), # C2 max
      (0, 4000), # C3 max
      (0, 1000), # C4 max
      (0, float("inf")), # A sem restricoes
      (0, float("inf")), # B sem restricoes
      (0, float("inf")) # C sem restricoes
      ] 
# Optimizacao por Simplex ++ 
opt = linprog(c = Obj, A_ub = LD, b_ub = LE, integrality = 1)
print(f"A:{opt.x[0]} B:{opt.x[1]}, Lucro: {opt.fun}")

ValueError: Invalid input for linprog: A_ub must have exactly two dimensions, and the number of columns in A_ub must be equal to the size of c

In [ ]:
# b)

Obj = [13, 15.30, 14.60, 14.90, # C1, C2, C3, C4
       -16.20, -15.75, -15.30] # A, B, C

# Inequacoes
# lado direito
LD = [[0.85, -0.15, -0.15, -0.15, 0, 0, 0], # inequacao 1 75% * C1 - 15% * C2 - 15% * C3 - 15% * C4 <= 0 
      [0.4, -0.6, 0.4, 0.4, 0, 0, 0], # inequacao 2 40% * C1 - 60% * C2 + 40% * C3 + 40% * C4 <= 0
      [-0.5, -0.5, 0.5, -0.5, 0, 0, 0], # inequacao 3 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      [-0.5, -0.5, 0.5, -0.5, 0, 0, 0], # inequacao 4 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      [0.9, -0.1, -0.1, -0.1, 0, 0, 0], # inequacao 5 90% * C1 - 10% * C2 + 10% * C3 - 10% * C4 <= 0 
      [0.1, -0.9, 0.1, 0.1, 0, 0, 0], # inequacao 6 10% * C1 - 90% * C2 + 10% * C3 + 10% * C4 <= 0
      [0.8, -0.2, -0.2, -0.2, 0, 0, 0] # inequacao 7 80% * C1 - 20% * C2 - 20% * C3 - 20% * C4 <= 0 
      ] 
# lado esquerdo
LD = [0, # inequacao 1 85% * C1 - 15% * C2 - 15% * C3 - 15% * C4 <= 0 
      0, # inequacao 2 40% * C1 - 60% * C2 + 40% * C3 + 40% * C4 <= 0
      0, # inequacao 3 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      0, # inequacao 4 -50% * C1 - 50% * C2 + 50% * C3 - 50% * C4 <= 0 
      0, # inequacao 5 90% * C1 - 10% * C2 + 10% * C3 - 10% * C4 <= 0 
      0, # inequacao 6 10% * C1 - 90% * C2 + 10% * C3 + 10% * C4 <= 0
      0 # inequacao 7 80% * C1 - 20% * C2 - 20% * C3 - 20% * C4 <= 0 
      ] 
# Contorno
CC = [(0, 3000), # C1 max 
      (0, 2000), # C2 max
      (0, 4000), # C3 max
      (0, 1000), # C4 max
      (0, float("inf")), # A sem restricoes
      (0, float("inf")), # B sem restricoes
      (0, float("inf")) # C sem restricoes
      ] 
# Optimizacao por Simplex ++ 
opt = linprog(c = Obj, A_ub = LD, b_ub = LE, integrality = 1)
print(f"A:{opt.x[0]} B:{opt.x[1]}, Lucro: {opt.fun}")

ValueError: Invalid input for linprog: A_ub must have exactly two dimensions, and the number of columns in A_ub must be equal to the size of c